#1) PIP Commands

In [ ]:
!pip install flask-ngrok
!pip install flask_wtf
!pip install python-bidi
!pip install --upgrade arabic-reshaper
!pip install tensorflow
!pip install keras
!pip install --user -U nltk
!pip install --user -U numpy
!pip install pyarabic
!pip install sentence_transformers
!pip install transformers
!pip install translate-toolkit
!pip install sentence_transformers

# Ngrok Server client authentificatioçn
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
!tar -xvf /content/ngrok-stable-linux-amd64.tgz
!./ngrok authtoken 2Er5sHdMaEwPpSxmspDdO0cR9Rr_276BLXZakSeFRZ46nnDhZ

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 136 kB 4.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 4.0 M

#2) Imports + Global Variables

##2.1) Import packages

In [ ]:
from google.colab import drive

# usual Packages
import numpy as np
import re
import sys
import os
import gc

from sklearn.utils import shuffle # To randomize dataset lines
from math import floor 
import pyarabic.araby as araby    # Using araby to remove diacritics in Arabic

# Natural Language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

# Ginsim module (Word2Vec)
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import remove_stopwords

# Keras module (LSTM and GRU models)
import tensorflow as tf
from keras.layers import Dense, LSTM, GRU, Embedding, Dropout, Flatten, Input
from keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from keras import callbacks
from keras.losses import CosineSimilarity

# Sentence transformer module (BERT Model)
from sentence_transformers import SentenceTransformer, models, InputExample, losses, util, evaluation,SentencesDataset # BERT Model
from torch import nn, cuda # So we can use GPU
import torch
from torch.utils.data import DataLoader #Torch data formater, works with BERT Model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

##2.2) Init console

In [ ]:
list_events_test=list()
def consoleln(text, color='white'):
  global list_events_test, allow_colab_print
  list_events_test.append("<span style='color:"+color+";'>"+" "+text+"</span><br>")
  if allow_colab_print:
    print(text)

def console(text, color='white'):
  global list_events_test
  list_events_test.append("<span style='color:"+color+";'>"+" "+text+"</span>")
  if allow_colab_print:
    print(text+" ", end='')

def consolesep():
  consoleln("------------------------------------------------------------------------------------------------------", 'gray')

def consolenext():
  console("-------------------------------------------------------", 'cyan')
  console("New Execution")
  consoleln("-----------------------------------------------------", 'cyan')

##2.3) Global vars, Better run this only once

In [ ]:
# Deep learning globals
w2v_model = Word2Vec()                  # Current w2v model
dl_model = Sequential()                 # Current deep learning model (Case Lstm/GRU)
dl_model_rev = Sequential()             # Reverse model of dl_model (Case Lstm/GRU)
dl_model_bert = SentenceTransformer()   # Current deep learning model (BERT)

custom_modeltype = ''                    # User's custom model type 'lstm_gru' <-> 'bert'
custom_nn = Sequential()                 # User's custom NN Ar->En (LSTM/GRU)
custom_nn_reverse = Sequential()         # User's custom NN En->Ar (LSTM/GRU)
custom_nn_bert = SentenceTransformer()   #User's custom NN (BERT)

# Datasets
data = None                             # STS dataset
split_ratio = 80                        # Split between traindata and testdata, it is changeable depending on the model selected
user_dataset = None                     # User's instance of STS dataset

# Duplication detectors, so we dont have to do the same long operations. BAD IDEA TO MESS WITH THIS GROUPE
bert_embeddings_created = False         # Check wether we created BERT's Embeddings
bert_embeddings_ar: list()              # Global bert embeddings for dataset sts
bert_embeddings_en: list()              # Same as above
dataset_loaded = False                  # not yet used
current_word2vec_name=''                # Check if we already loaded the same spesific Word2Vec
var_radio_dl_model=''                   # Check the selected radio button if we need to load a new DL Model
var_radio_dl_model_old=''               # Is used to compare with the current selected DL Model
old_dl_model_name=''                    # Same as 'var_radio_dl_model' but is used in coding instead of flask interface
custom_nn_model = False                 # Using this to display special interface for testing user's custom model

# Misc.
maxlength=20                            # We define the minimum 'maxlength' for our models that are already created
list_events_test=list()                 # this list will be written in console
first_run = True                        # Make sure to reinint console after first main run
allow_colab_print = True                # Self-explanatory


#3) Mount Drive

In [ ]:
# Mail:       pfewalidhocine2022@gmail.com
# PassWord:   PFEWALIDHOCINE1999

drive.mount('/content/drive') 
os.chdir('/content')

console("Montage du")
console("'Google Drive'", 'green')
console("pour avoir accès au dataset et différents modeès")
console("Word2Vec AR-EN", 'pink')
console(", et des diferent modèles entrainés de deelplearning")
consoleln("(LSTM, GRU, et BERT)", 'pink')
consolesep()


Mounted at /content/drive
Montage du 'Google Drive' pour avoir accès au dataset et différents modeès Word2Vec AR-EN , et des diferent modèles entrainés de deelplearning (LSTM, GRU, et BERT)
------------------------------------------------------------------------------------------------------


#4) Load Stuff

##4.1) Load STS dataset

In [ ]:
# Read dataset + split into trainset/testset
class DatasetLoader:
  def __init__(self):
    self.lines_en = list()
    self.lines_ar = list()
    self.lines_score = list()
    self.train_en = list()
    self.train_ar = list()
    self.train_score = list()
    self.test_en = list()
    self.test_ar = list()
    self.test_score = list()
    self.size = 0
    self.train_size = 0
    self.test_size = 0

  def load(self, split_ratio, random = False):
    print('Loading dataset ', end='', sep='')
    with open("/content/drive/MyDrive/Colab Notebooks/dataset.txt", "r") as dataset_file:
      for line in dataset_file:
        seg = line.split("\t")
        seg[0] = re.sub(r'[^\w]', ' ', seg[0])
        seg[1] = re.sub(r'[^\w]', ' ', seg[1])
        seg[1] = araby.strip_diacritics(seg[1])
        self.lines_en.append(seg[0])
        self.lines_ar.append(seg[1])
        #print("taille lines ar",len(lines_ar))
        #print("taille lines en",len(lines_en))
    # Read scores
    with open("/content/drive/MyDrive/Colab Notebooks/dataset_score.txt", "r") as score_file:
      for line in score_file:
        self.lines_score.append(float(line.strip()))

    split_value =  floor(split_ratio*len(self.lines_ar))
    if random:
      print('with shuffling...')
      temp_en, temp_ar, temp_sc= shuffle(self.lines_en,self.lines_ar, self.lines_score)
      self.train_en = temp_en[:split_value]
      self.train_ar = temp_ar[:split_value]
      self.train_score = temp_sc[:split_value]
      self.test_en = temp_en[split_value:]
      self.test_ar = temp_ar[split_value:]
      self.test_score = temp_sc[split_value:]
      self.size = len(temp_en)
      self.train_size = len(self.train_en)
      self.test_size = len(self.test_en)
    else:
      print('without shuffling...')
      self.train_en = self.lines_en[:split_value]
      self.train_ar = self.lines_ar[:split_value]
      self.train_score = self.lines_score[:split_value]
      self.test_en = self.lines_en[split_value:]
      self.test_ar = self.lines_ar[split_value:]
      self.test_score = self.lines_score[split_value:]
      self.size = len(self.lines_en)
      self.train_size = len(self.train_en)
      self.test_size = len(self.test_en)

try:
    del data
except NameError:
  pass

#list_events_test.append("Debut de lecture du dataset STS avec découpage entre les données training/test = "+str(split_ratio)+"%...")
data = DatasetLoader()
data.load(split_ratio/100, False)

console("Fin de lecture de dataset")
consoleln("STS.",'green')

console("Taille de la list du dataset =")
consoleln(str(data.size)+" lignes", 'pink')

console("Taille du données d\'apprentissage = ")
consoleln(str(data.train_size)+" lignes", 'pink')

console("Taille du données de tests = ")
consoleln(str(data.test_size)+" lignes", 'pink')

consolesep()

Loading dataset without shuffling...
Fin de lecture de dataset STS.
Taille de la list du dataset = 250 lignes
Taille du données d'apprentissage =  200 lignes
Taille du données de tests =  50 lignes
------------------------------------------------------------------------------------------------------


##4.2) Gensim Word2Vec

In [ ]:
def load_w2v_model(w2v_model_name):
  global w2v_model, current_word2vec_name, list_events_test
  if current_word2vec_name !=w2v_model_name:
    console("Chargement d\'un nouveau modèle")
    console("Word2Vec AR-EN", 'green')
    consoleln("à partir du Google Drive, cela peut prende un moment...")
    console("le nom code du modèle est:")
    console(str(w2v_model_name), "pink")
    current_word2vec_name = w2v_model_name

    os.chdir('/content/drive/My Drive/ArbEngVec gensim/')
    if w2v_model_name == 'SkipGram_ParaAlignement':
      w2v_model = Word2Vec.load("ParallelAlign_Skip/nonshuffle_5window_skipgram_300size.model")
    elif w2v_model_name == 'SkipGram_WordByWord':
      w2v_model = Word2Vec.load("WordByWord_Skip/wordbywordshuffle_5window_skipgram_300size.model")
    elif w2v_model_name == 'SkipGram_RandomShuffle':
      w2v_model = Word2Vec.load("RandomShuffle_Skip/randshuffle_5window_skipgram_300size.model")
    elif w2v_model_name == 'CBOW_ParaAlignement':
      w2v_model = Word2Vec.load("ParallelAlign_CBOW/nonshuffle_5window_cbow_300size.model")
    elif w2v_model_name == 'CBOW_WordByWord':
      w2v_model = Word2Vec.load("WordByWord_CBOW/wordbywordshuffle_5window_cbow_300size.model")
    elif w2v_model_name == 'CBOW_RandomShuffle':
      w2v_model = Word2Vec.load("RandomShuffle_CBOW/randshuffle_5window_cbow_300size.model")
    else:
      raise Exception("Incorrect Word2Vec model name")
      consoleln("Le nom du modèle Word2Vec est incorrect!", 'red')
      consoleln("Le nouveau modèle Word2Vec à été téléchargé avec succès!", 'green')
  else:
    consoleln("Le même modèle Word2Vec est déjà chargé, pas besoin de le télécharger à nouveau!", 'yellow')
  os.chdir('/content')
  gc.collect()
  return

load_w2v_model('SkipGram_WordByWord')

Chargement d'un nouveau modèle Word2Vec AR-EN à partir du Google Drive, cela peut prende un moment...
le nom code du modèle est: SkipGram_WordByWord 

##4.3) Load our own trained LSTM/GRU/BERT

In [ ]:
def load_deep_learning_model_pre(dl_model_name):
  global dl_model, dl_model_rev, dl_model_bert, bert_embeddings_created, list_events_test, old_dl_model_name
  consoleln("chargement d'un nouveau modèle de deep learning pré-entrainé par nous...")
  console("le nom code du modèle de deep learning est:")
  consoleln(str(dl_model_name), "pink")
  os.chdir('/content/drive/My Drive/Colab Notebooks/Web/pre_entrainer')

  if old_dl_model_name == dl_model_name:
    consoleln("Le modèle deep learning n'a pas été changé, en garde le modèle ancien...", 'yellow')
    return
  else:
    consoleln("Le modèle deep learning a  été changé, Cela va prendre du temps pour chargé ce que vous avez choisi...", 'yellow')
  
  dl_models_list={
    # -- LSTM --            
    'LSTM_1':           'LSTM_300_RandomGensim_3_couche_500_epoch_50_tanh_batch_1_ar_en_75%.h5', 
    'LSTM_1rev':        'LSTM_300_RandomGensim_3_couche_500_epoch_50_tanh_batch_1_en_ar_75%.h5', 

    'LSTM_2':           'LSTM_300_RandomGensim_3_couche_500_epoch_50_tanh_batch_4_ar_en_75%.h5', 
    'LSTM_2rev':        'LSTM_300_RandomGensim_3_couche_500_epoch_50_tanh_batch_4_en_ar_75%.h5', 
    
    'LSTM_3':           'LSTM_300_1_couche_500_epoch_50_sigmoid_ar_en.h5',
    'LSTM_3rev':        'LSTM_300_1_couche_500_epoch_50_sigmoid_en_ar.h5',

    # -- Bi. LSTM --
    'LSTM_B1':          'Bidi_LStm_2couche_dropout(0.2_0.3)_sigmoid_Epoch250_data70%_ar_en_accuracy24%_.h5',
    'LSTM_B1rev':       'Bidi_LStm_2couche_dropout(0.2_0.3)_sigmoid_Epoch250_data70%_en_ar_accuracy23%_.h5',

    'LSTM_B2':          'sp_Bid_LSTM300_RandomGensim300__2couche340_epoch300_tanh_dropout_batch_1_ar_en_resultat_18_48%_data_75%.h5',
    'LSTM_B2rev':       'sp_Bid_LSTM300_RandomGensim300__2couche340_epoch300_tanh_dropout_batch_1_en_ar_resultat_20%_data_75%.h5',

    'LSTM_unused1':     '',
    'LSTM_unused1rev':  'Bid_LSTM300_RandomGensim300__2couche340_epoch300_tanh_batch_1_en_ar_resultat_23.5%_data_75%.h5',

    # -- GRU --
    'GRU_1':            'GRU_300_RandomGensim300_3couche400_epoch750_tanh_batch20_dropout0.1_ar_en_accuracy19%_data75%.h5',  
    'GRU_1rev':         'GRU_300_RandomGensim300_3couche400_epoch750_tanh_batch20_dropout0.1_en_ar_accuracy14%_data75%.h5',  

    'GRU_2':            'GRU_300_RandomGensim300_1couche500_epoch250_tanh_batch20_ar_en_accuracy17%_data75%.h5',
    'GRU_2rev':         'GRU_300_RandomGensim300_1couche500_epoch250_tanh_batch20_en_ar_accuracy18%_data75%.h5',
    
    'GRU_3':            'GRU_300_RandomGensim300_1couche500_epoch250_tanh_batch20_dropout0.1_ar_en_accuracy14%_data75%.h5',
    'GRU_3rev':         'GRU_300_RandomGensim300_1couche500_epoch250_tanh_batch20_dropout0.1_en_ar_accuracy13%_data75%.h5',

    # -- Bi. GRU --
    'GRU_B1':           'N_bidiGRU_ar_en_accuracy15%.h5',# details dans le  memoire
    'GRU_B1rev':        'N_bidiGRU_en_ar_accuracy15_68%.h5',
    
    'GRU_B2':           'bidi_GRU400_RandomGensim300_3couche400_epoch250_tanh_batch10_dropout(0.2)_ar_en_accuracy19%__data75%.h5',   
    'GRU_B2rev':        'bidi_GRU400_RandomGensim300_3couche400_epoch250_tanh_batch10_dropout(0.2)_en_ar_accuracy21%__data75%.h5',
    
    'GRU_B3':           'Bid_GRU300_RandomGensim300__1couche500_epoch300_tanh_batch_2_ar_en_resultat_20%_data_75%.h5',
    'GRU_B3rev':        'Bid_GRU300_RandomGensim300__1couche500_epoch300_tanh_batch_4_en_ar_resultat_21%_data_75%.h5',

    'GRU_B4':           'bidi_GRU400_RandomGensim300_2couche400_epoch250_tanh_batch1_ar_en_accuracy21%__data75%.h5',
    'GRU_B4rev':        'bidi_GRU400_RandomGensim300_2couche400_epoch250_tanh_batch1_en_ar_accuracy23%_data75%.h5',

    'GRU_unusedB5':     'bidi_GRU400_RandomGensim300_2couche400_epoch250_tanh_batch10_ar_en_accuracy20%__data75%.h5',
    'GRU_unusedB5rev':  '',

    # -- BERT --
    'BERT_A':           'modelbert_100ep_batch30.h5',
    'BERT_B':           'modelbert_10ep_batch20.h5',
    'BERT_C':           'modelbert_50ep_batch40.h5'
  }

  if dl_model_name.startswith('BERT'):
    dl_model_bert = SentenceTransformer(dl_models_list[dl_model_name]+'/')
    bert_embeddings_created = False
    dl_model = Sequential()
    dl_model_rev = Sequential()
    gc.collect()
    old_dl_model_name = dl_model_name
    consoleln("le Modèle deep learning a été changé avec success!", 'green')
    return

  else: #LSTM/GRU

    try:
      dl_model = load_model(dl_models_list[dl_model_name])
    except OSError:
      consoleln('Le Modèle Deep learningne peut pas etre trouvé!!!, Erreur chemin', "red")
      raise('Model File path not found!!')
    except ValueError:
      try:
        dl_model = load_model(dl_models_list[dl_model_name], custom_objects={"f1_m": 'f1_m', 'precision_m':'precision_m', 'recall_m':'recall_m'})
      except OSError:
        consoleln('Le Modèle Deep learningne peut pas etre trouvé!!!, Erreur chemin', "red")
        raise('Model File path not found!!')

    #Again
    try:
      dl_model_rev = load_model(dl_models_list[dl_model_name+'rev'])
    except OSError:
      consoleln('Le Modèle Deep learningne peut pas etre trouvé!!!, Erreur chemin', "red")
      raise('Model File path not found!!')
    except ValueError:
      try:
        dl_model_rev = load_model(dl_models_list[dl_model_name+'rev'], custom_objects={"f1_m": 'f1_m', 'precision_m':'precision_m', 'recall_m':'recall_m'})
      except OSError:
        consoleln('Le Modèle Deep learningne peut pas etre trouvé!!!, Erreur chemin', "red")
        raise('Model File path not found!!')
    
    dl_model_bert = SentenceTransformer()
    gc.collect()
    old_dl_model_name = dl_model_name
    consoleln("le Modèle deep learning a été changé avec success!", 'green')
    return


#5) Utils functions

##5.1) First Set: Preprocess STS Dataset

In [ ]:

# Returns number of words in the longest sentence exists
def get_max_length(text_list):
    max_length = 0
    for string in text_list:
        if len(string.split(" ")) > max_length:
            max_length = len(string.split(" "))
    return max_length


# Replaces some unwanted caracters from sentences
def preprocess_string(string:str):
  string = string.lower()
  string = string.replace('/', ' ').replace('-', ' ').replace('.', ' ')
  string = araby.strip_diacritics(string)
  return string


# Removes escape caracters from words
def remove_escape_caracters(word_any:str):
  word_any = re.sub('^\s+', '', word_any)
  return re.sub('\.|,|\n|،|/|-', '', word_any)


# Replaces some caracters in arabic words
def rewrite_arabic_word(word_ar:str):
  return word_ar.replace('أ', 'ا').replace('إ', 'ا').replace('آ', 'ا').replace('ؤ', 'و').replace('ئ', 'ى').replace('ة', 'ه').replace('ي', 'ى').replace(' ّ', '').replace('اً', 'ا')


# Trying to removes some affixes (if present) from a word in case of catch
def remove_affix_lv1(word_any:str): 
  if word_any.startswith('بال') or word_any.startswith('لال') or word_any.startswith('فال')  or word_any.startswith('كال') or word_any.startswith('وال'):
    return word_any[1:]
  if word_any.endswith('s'):
    return word_any[:len(word_any)-1]
  if word_any.endswith('ing'):
    return word_any[:len(word_any)-3]
  return word_any #if nothing resturns as it is


# Same thing, for catch n°2
def remove_affix_lv2(word_any:str): # in case the word falls in catch lvl 2
  if word_any.startswith('بل') or word_any.startswith('لل') or word_any.startswith('فل')  or word_any.startswith('كل') or word_any.startswith('ول'):
    return word_any[2:]
  if word_any.startswith('ال'):
    return word_any[2:]
  return word_any


# catch n°3
def remove_affix_lv3(word_any:str): # in case the word falls in catch lvl 3
  liste_1n = ['ا','ي','ت','ك','و','ة','ه']
  liste_2n = ['ات','كم','ما','ون','تم','نا','ني','ين','وه','يه','ته','ها','هم','هن','يا','ان','ية','وا', 'ly', 'er']
  liste_3n = ['بات','تان','ونن','اته','ونه','تنا','وها','يها','تها','وهم','يون','اها','تين','يين','تهم','هما', 'ful', 'ing']
  liste_4n = ['انكم','يتنا','اتنا','يتها','اتها','اتية','اتهم', 'ness', 'able', 'less', 'ment']
  liste_5n = ['تموها']

  if word_any.endswith('ier') or word_any.endswith('ily'): 
    return word_any[:len(word_any)-3] + 'y'
  if word_any.endswith('iness'):
    return word_any[:len(word_any)-4] + 'y'

  for endings in liste_1n + liste_2n + liste_3n + liste_4n + liste_5n:
    if word_any.endswith(endings):
      return word_any[:len(word_any)-len(endings)]
  return word_any #if cant do anything


# Coding list of string into list of w2v vectors
# 1: take one string from the list and tokenizing it into list of words
# 2: remove common words
# 3: take first word & try to get its w2v vector from a model
# 4: if fails, try to modify the word and try again
# 5: jump to next word & repeat 3
# 6: jump to next string & repeat 1
def word2vec_encoding(list_sentences):
    global w2v_model, list_events_test
    if list_sentences == []:
      return []
    encoded_list_sentences = list()
    encoded_sentence = list()
    consoleln("Chargement de la liste des mots vides du package NLTK pour la langue spécifique...")
    stop_words_en = set(stopwords.words('english'))
    stop_words_ar = set(stopwords.words("arabic"))
    consoleln("Création d\'une list des mot vides customisé qui contient des nouveaux mots vides...")
    stop_words_custom =['a', '\'re', '\'s', '']

    nbr_common_found = 0
    nbr_word_non_valid = 0
    nbr_tatal_words = 0

    consoleln("Début de processus de création des embeddings...")
    consoleln("Le processus est:")
    consoleln("----> 1-: Prendre une phrase de la liste et la faire tokenisé en list des mots", "gray")
    consoleln("----> 2-: Elimination des mot vides", "gray")
    consoleln("----> 3-: Prendre un mot de la list des tokens et essayer de trouver son vecteur embeddings en utilisant Word2Vec AR-EN", "gray")
    consoleln("----> 4-: Si on échoue, on va prendre le mot et lui appliqué un ensemble de modification de niveau 1 (suppression des préfixes et suffixes)", "gray")
    consoleln("----> 5-: Réessayer de trouver le vecteur du nouveau mot changé", "gray")
    consoleln("----> 6-: Si on échoue, on va appliqué deux autre ensembles des modifications (niveau 2 et 3) (suppression des préfixes et suffixes qui ne sont pas commune et changement des lettres)", "gray")
    consoleln("----> 7-: Si on échoue encore, on n\'a pas pu trouvé le vecteur du mot dans notre word embeddings Word2Vec AR-EN)", "gray")
    consoleln("----> 8-: Le mot va être oublié et on va prendre le prochain mot de la list des tokens puis repéter (3)", "gray")
    consoleln("----> 9-: Si on termine la codification des tokens, les embeddings sont mit dans une liste qui represent la phrase courante", "gray")
    consoleln("----> 10-: Prendre la phrase suivante et repéter (1)", "gray")
    consoleln("----> 11-: à la fin de codification de tout les phrases, les listes des embeddings généré sont mit dans une liste globale", "gray")

    for sentence in list_sentences:
        sentence = preprocess_string(sentence)
        tokens = word_tokenize(sentence)
        encoded_sentence = list()
        words = list()
        nbr_tatal_words += len(tokens)
        for word in tokens:
          word = remove_escape_caracters(word)
          word = rewrite_arabic_word(word)
          if word not in stop_words_en and word not in stop_words_ar and word not in stop_words_custom:
            words.append(word)
          else:
            nbr_common_found += 1
        for word in words:
          try:
            encoded_sentence.append(w2v_model.wv[word]) 
          except KeyError:
            #print('word at lvl1 before [', word, ']', sep='', end='')
            word = remove_affix_lv1(word)
            #print('\tword at lvl1  after [', word, ']', sep='')
            try:
              encoded_sentence.append(w2v_model.wv[word])
            except KeyError:
              #print('word at lvl2 before [', word, ']', sep='', end='')
              word = remove_affix_lv2(word)
              #print('\tword at lvl2 after [', word, ']', sep='')
              try:
                encoded_sentence.append(w2v_model.wv[word])
              except KeyError:
                #print('word at lvl3 before [', word, ']', sep='', end='')
                word = remove_affix_lv3(word)
                #print('\tword at lvl3 after [', word, ']', sep='')
                try:
                  encoded_sentence.append(w2v_model.wv[word])
                except KeyError:
                  nbr_word_non_valid += 1
                  #print('invalid word [', word, ']\n', sep='')
        encoded_list_sentences.append(encoded_sentence)
    
    consoleln("Fin du processus, les resultats sont:", 'green')
    console("Le nombre des mots totals de la langue courante sont:           ")
    consoleln(str(nbr_tatal_words), 'pink')
    console("Le nombre des mots valides de la langue courante sont:          ")
    consoleln(str(nbr_tatal_words - nbr_common_found - nbr_word_non_valid), 'pink')
    console("Le nombre des mots invalides de la langue courante sont:        ")
    consoleln(str(nbr_word_non_valid), 'pink')
    console("Le nombre des mots vides supprimés de la langue courante sont:  ")
    consoleln(str(nbr_common_found), 'pink')
    consolesep()
    return encoded_list_sentences
  
# Adding zeors to the end of coded sentences to reach max length, (in the end they'll all have the same length)
def add_paddings(encoded_list_sentences, max_length):
  padded_encoded_list_sentences = list()
  for encoded_sentence in encoded_list_sentences:
      zero_padding_cnt = max_length - len(encoded_sentence)
      pad = np.zeros((1, 300))
      for i in range(zero_padding_cnt):
          encoded_sentence = np.concatenate((encoded_sentence, pad), axis=0)
      padded_encoded_list_sentences.append(encoded_sentence)
  return padded_encoded_list_sentences


def preprocess(lines_en, lines_ar):
  global maxlength, list_events_test
  maxlength_en = get_max_length(lines_en)
  maxlength_ar = get_max_length(lines_ar)
  console('comparaison de nombre des mots de la plus grande phrase dans la liste des phrase')
  console('d\'anglais ('+str(maxlength_en)+')', 'pink')
  console("avec celui de la liste ")
  console("d\'arabe ("+str(maxlength_ar)+")", 'pink')
  console(' Et avec le Mexlength minimum défini par ')
  console("nous ("+str(maxlength)+")", 'pink')

  console('cette comparaison va nous données une idéé sur la taille de paddings qu\'on va l\'utilisé après')
  maxlen = max(maxlength_en, maxlength_ar, maxlength)
  console('Le (MaxLength) a été choisi ('+str(maxlen)+')', 'pink')
  consoleln(', Les phrases seront étendues avec des paddings jusqu\'ils sont tousde même taille.')
  consolesep()

  console("debut de la codification des mots de la list des phrases Anglais pour avoir des embeddings en utilisant")
  console("Word2Vec", "green")
  consoleln("...")
  encoded_en = word2vec_encoding(lines_en)
  consoleln("fin de la codification de la list d\'anglais.", 'green')
  consolesep()

  console("debut de la codification des mots de la list des phrases Arabe pour avoir des embeddings en utilisant")
  console("Word2Vec", "green")
  consoleln("...")
  encoded_ar = word2vec_encoding(lines_ar)
  consoleln("fin de la codification de la list d\'arabe.", 'green')
  consolesep()

  console("Utilisation du")
  console("MaxLength", 'pink')
  consoleln("pour faire ajouter les paddings au embeddings créés")
  padded_encoded_en = add_paddings(encoded_en, maxlength)
  padded_encoded_ar = add_paddings(encoded_ar, maxlength)
  consoleln("Ajpout des paddings terminé!", 'green')

  embeddings_en = np.array(padded_encoded_en)
  embeddings_ar = np.array(padded_encoded_ar)
  return embeddings_en, embeddings_ar


##5.2) Second Set: Similarity & Evaluation

In [ ]:
def calc_cos_similarity_lstm_gru(lstm_gru_output, top_n):
  global data, list_events_test
  cosine_loss = tf.keras.losses.CosineSimilarity(axis = -1) 

  consoleln("Debut de calcul des similarités entre l\'input de l\'utilisateur avec le reste du dataset...")
  consoleln("La codification du dataset va subir les même prétraitement pour cération des embeddings...", 'yellow')
  dataset_embed_en, dataset_embed_ar = preprocess(data.lines_en, data.lines_ar)
  consoleln("Fin de la codification des donnés de test", "green")

  lstm_gru_output = lstm_gru_output.astype('double')
  cos_list, result_list, most_similaire = list(), list(), list()

  consoleln("Debut de calcul de mésure de similarité Cosinus entre l\'input avec tout les dataset...")
  for i in range(len(data.lines_ar)):
    cos_list.append(abs(cosine_loss(dataset_embed_ar[i], lstm_gru_output).numpy()))
  for i in range(len(data.lines_en)):
    cos_list.append(abs(cosine_loss(dataset_embed_en[i], lstm_gru_output).numpy()))

  most_similaire = sorted(zip(cos_list, data.lines_ar+data.lines_en), reverse = True)
  for i in range(top_n):
    result_list.append((most_similaire[i][1].strip(), float("{:.4f}".format(most_similaire[i][0]))))
    
  consoleln("Fin de calcul de mésure de similarité", 'green')
  return result_list



def evaluate_model_keras(user_input, top_n, langue, custom):
  global dl_model, dl_model_rev, w2v_model, list_events_test, custom_nn, custom_nn_reverse
  if langue=='ar':
    consoleln("Basculer vers le model qui accespt la langue Arabe comme entrée",'yellow')
    if custom:
      temp_model = custom_nn
    else:
      temp_model = dl_model
  else:
    consoleln("Basculer vers le model qui accespt la langue Anglais comme entrée",'yellow')
    if custom:
      temp_model = custom_nn_reverse
    else:
      temp_model = dl_model_rev

  gc.collect()
  consoleln("Pour fair l'évaluation la données d'entré d\'utilisateur va être d\'abord codifié en word embeddings...")
  embedded_user_input, _ = preprocess([user_input],[])
  consoleln("La codification de la données d\'utilisateur a été terminé!", 'green')

  consoleln("Le modèle de deep learning courant va essayer de predire le resultat en lui passant les embeddings d\'utilisteur une par une. La prediction va générer les embeddings une par une aussi")
  y = temp_model.predict(embedded_user_input)
  consoleln("La prediction a été terminé, nous avons l\'embeddings résultat!", "green")

  consoleln("Un jeux de test (à ne pas prendre en considiration)","gray")
  consoleln("On va essayé de tourves les embeddings proches (de vrai mots) pour les embeddings prédit par le modèle deep learning courant:","gray")
  for i in range(y.shape[1]):
    consoleln("Le mot proche d\'embeddings généré n°"+str(i+1)+" est: "+str(w2v_model.wv.most_similar(positive=y[0,[i]], topn=1)),"gray")
  return calc_cos_similarity_lstm_gru(y, top_n)



def evaluate_model_bert(user_input, top_n, custom):
  from sentence_transformers import SentenceTransformer, util
  global dl_model_bert, bert_embeddings_created, bert_embeddings_ar, bert_embeddings_en, custom_nn_bert, data
  if custom:
    dl_model_bert = custom_nn_bert
    gc.collect()
    bert_embeddings_created = False

  # Compute embedding for both lists
  if not bert_embeddings_created or custom:
    consoleln("Les embeddings du dataset pour le modèle BERT courant ne sont pas créées, on va les créé maintenant", 'yellow')
    
    bert_embeddings_ar = dl_model_bert.encode(data.lines_ar, convert_to_tensor=True)
    bert_embeddings_en = dl_model_bert.encode(data.lines_en, convert_to_tensor=True)
    bert_embeddings_created = True
    consoleln("Fin de creation des embeddings!", 'green')
    gc.collect()
  else:
    consoleln("Les embeddings du dataset pour le modèle BERT existent deja, pas la peine de les créer à nouveau", 'yellow')

  # Embed user's input  
  consoleln("Codification du l'entré de l'utilisateur en embeddings", 'yellow')
  bert_user_embeddings = dl_model_bert.encode(user_input, convert_to_tensor=True)

  # Compute cosine similarities
  consoleln("Calcule de la similarité entre les embeddings de dataset avec celui de l'utilisateur", 'yellow')
  cosine_scores_ar = util.cos_sim(bert_embeddings_ar, bert_user_embeddings)
  cosine_scores_en = util.cos_sim(bert_embeddings_en, bert_user_embeddings)

  # Concat two tensors results ar + en
  concated = torch.cat((cosine_scores_ar, cosine_scores_en))

  # Sort while keeping in mind swaping the order of dataset lines too
  cosine_scores = sorted(zip(concated, data.lines_ar+data.lines_en), reverse = True)
  
  consoleln("Normalisation les valeurs de similartés entre [0,1] et les faire le trié selon l'odre decroissant", 'yellow')
  # Normalizing similarity into [0, 1] instead of [-1, 1] & rewriting evrything into a good looking list
  cosine_scores = [(y.strip(), float("{:.4f}".format((x[0].item()+1)/2))) for x,y in cosine_scores]

  # Assertion check
  for x in cosine_scores:
    assert float(x[1])>=0 and float(x[1]) <=1
  
  return cosine_scores[0:top_n]




##5.3) Third Set: Flask Route Functions Helpers

In [ ]:
def parse_request(request):
  nn_model_type = ''
  w2v_model_type = ''
  nbr_layers = ''
  nbr_in_layer1 = ''
  nbr_in_layer2 = ''
  nbr_in_layer3 = ''
  nbr_in_layer4 = ''
  nbr_in_layer5 = ''
  dense = ''
  weights = ''
  dropout = ''
  activ_fun = ''
  learn_rate = ''
  momentum = ''
  decay_rate = ''
  loss_fun = ''
  epochs = ''
  batchsize = ''
  optim = ''
  split_ratio = ''
  embeddings_length = ''

  nn_model_type = request.form['radgroup2']
  if nn_model_type != 'BERT':
    w2v_model_type = request.form['radgroup']
  if nn_model_type == 'LSTM':
    nbr_layers = request.form['nodesnumber_lstm']
    nbr_in_layer1 = request.form['lstm_nbr_units_1']
    nbr_in_layer2 = request.form['lstm_nbr_units_2']
    nbr_in_layer3 = request.form['lstm_nbr_units_3']
    nbr_in_layer4 = request.form['lstm_nbr_units_4']
    nbr_in_layer5 = request.form['lstm_nbr_units_5']
    dense =  request.form['densesnumber_lstm']
    weights = request.form['weights_lstm']
    dropout = request.form['dropout_lstm']
    activ_fun = request.form['activ_fun_lstm']
    learn_rate = request.form['Learn_rate_lstm']
    momentum = request.form['momentum_lstm']
    decay_rate = request.form['decay_lstm']
    loss_fun = request.form['loss_fun_lstm']
    epochs = request.form['nbrepoch_lstm']
    batchsize = request.form['batch_lstm']
    optim = request.form['optim_fun_lstm']
    split_ratio = request.form['split_ratio_lstm']
  elif nn_model_type == 'GRU':
    nbr_layers = request.form['nodesnumber_gru']
    nbr_in_layer1 = request.form['gru_nbr_units_1']
    nbr_in_layer2 = request.form['gru_nbr_units_2']
    nbr_in_layer3 = request.form['gru_nbr_units_3']
    nbr_in_layer4 = request.form['gru_nbr_units_4']
    nbr_in_layer5 = request.form['gru_nbr_units_5']
    dense =  request.form['densesnumber_gru']
    weights = request.form['weights_gru']
    dropout = request.form['dropout_gru']
    activ_fun = request.form['activ_fun_gru']
    learn_rate = request.form['Learn_rate_gru']
    momentum = request.form['momentum_gru']
    decay_rate = request.form['decay_gru']
    loss_fun = request.form['loss_fun_gru']
    epochs = request.form['nbrepoch_gru']
    optim = request.form['optim_fun_gru']
    batchsize = request.form['batch_gru']
    split_ratio = request.form['split_ratio_rgu']
  else: # BERT
    embeddings_length = request.form['embeddings_lenth_bert']
    activ_fun = request.form['actv_func_bert']
    loss_fun = request.form['loss_func_bert']
    epochs = request.form['nbrepoch_bert']
    split_ratio = request.form['split_ratio_bert']
    batchsize = request.form['batch_bert']

  if nn_model_type == 'LSTM' or nn_model_type == 'GRU':
    consoleln('-----------------------------------------------------------------------------------')
    consoleln('--------------------------------- Summary -----------------------------------------')
    consoleln('-----------------------------------------------------------------------------------')
    consoleln('NN model: '+nn_model_type)
    consoleln('W2V model: '+w2v_model_type)
    consoleln('Nbr Layers: '+nbr_layers)
    consoleln('Layer 1: '+nbr_in_layer1)
    consoleln('Layer 2: '+nbr_in_layer2)
    consoleln('Layer 3: '+nbr_in_layer3)
    consoleln('Layer 4: '+nbr_in_layer4)
    consoleln('Layer 5: '+nbr_in_layer5)
    consoleln('Dense: '+dense)
    consoleln('Weights: '+weights)
    consoleln('Dropout: '+dropout)
    consoleln('Activation: '+activ_fun)
    consoleln('Optimizer: '+optim)
    consoleln('Learn rate: '+learn_rate)
    consoleln('Momentum: '+momentum)
    consoleln('Decay rate: '+decay_rate)
    consoleln('Loss funct: '+loss_fun)
    consoleln('Epochs: '+epochs)
    consoleln('split ratio: '+split_ratio+"%")
    consoleln('Batch size: '+batchsize+"%")
    consoleln('-----------------------------------------------------------------------------------')
    consoleln('-----------------------------------------------------------------------------------')
  else:
    consoleln('-----------------------------------------------------------------------------------')
    consoleln('--------------------------------- Summary -----------------------------------------')
    consoleln('-----------------------------------------------------------------------------------')
    consoleln('NN model: '+nn_model_type)
    consoleln('Embeddings Length: '+embeddings_length)
    consoleln('Activation: '+activ_fun)
    consoleln('Loss funct: '+loss_fun)
    consoleln('Epochs: '+epochs)
    consoleln('split ratio: '+split_ratio+"%")
    consoleln('Batch size: '+batchsize+"%")
    consoleln('-----------------------------------------------------------------------------------')
    consoleln('-----------------------------------------------------------------------------------')
  return  nn_model_type,\
          w2v_model_type ,\
          nbr_layers,\
          nbr_in_layer1,\
          nbr_in_layer2,\
          nbr_in_layer3,\
          nbr_in_layer4,\
          nbr_in_layer5,\
          dense,\
          weights,\
          dropout,\
          activ_fun,\
          learn_rate,\
          momentum,\
          decay_rate,\
          loss_fun,\
          epochs,\
          batchsize,\
          optim,\
          split_ratio,\
          embeddings_length

def get_keras_loss_fun(loss_fun):
  if loss_fun == 'Mean Squared Error':
    return tf.keras.losses.MeanSquaredError()
  elif loss_fun == 'Mean Absolute Error':
    return tf.keras.losses.MeanAbsoluteError()
  elif loss_fun == 'Mean Squared Logarithmic Error':
    return tf.keras.losses.MeanSquaredLogarithmicError()
  elif loss_fun == 'Cosine Similarity Loss':
    return tf.keras.losses.CosineSimilarity(axis=1)
  elif loss_fun == 'LogCosh Loss':
    return tf.keras.losses.LogCosh()

def get_keras_optimizer(optim, lr_schedule, momentum) :
    if optim == 'SGD':
      return tf.keras.optimizers.SGD(learning_rate=lr_schedule, momentum=float(momentum))
    elif optim == 'RMSprop':
      return tf.keras.optimizers.RMSprop(learning_rate=lr_schedule, momentum=float(momentum))
    elif optim == 'Adam':
      return tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    elif optim == 'Adadelta':
      return tf.keras.optimizers.Adadelta(learning_rate=lr_schedule)
    elif optim == 'Adagrad':
      return tf.keras.optimizers.Adagrad(learning_rate=lr_schedule)
    elif optim == 'Adamax':
      return tf.keras.optimizers.Adamax(learning_rate=lr_schedule)
    elif optim == 'Nadam':
      return tf.keras.optimizers.Nadam(learning_rate=lr_schedule)
    elif optim == 'Ftrl':
      return tf.keras.optimizers.Ftrl(learning_rate=lr_schedule)

def get_bert_active_fun(activ_fun):
  if activ_fun == 'ReLU':
    return nn.ReLU()
  elif activ_fun == 'LeakyReLU':
    return nn.LeakyReLU()
  elif activ_fun == 'Sigmoid':
    return nn.Sigmoid()
  elif activ_fun == 'Tanh':
    return nn.Tanh()
  elif activ_fun == 'Softmax':
    return nn.Softmax()

def get_bert_loss_fun(bert_model, loss_fun):
  if loss_fun == 'ContrastiveLoss':
    return losses.ContrastiveLoss(model=bert_model)
  elif loss_fun == 'CosineSimilarityLoss':
    return losses.CosineSimilarityLoss(model=bert_model)
  elif loss_fun == 'SoftmaxLoss':
    return losses.SoftmaxLoss(model=bert_model)
  #
  #elif loss_fun == 'MSELoss':
  #  return losses.MSELoss(model=bert_model)
  #elif loss_fun == 'MarginMSELoss':
  #  return losses.MarginMSELoss(model=bert_model)
  #elif loss_fun == 'MultipleNegativesRankingLoss':
  #  return losses.MultipleNegativesRankingLoss(model=bert_model) # Recommended


#6) Flask App

In [ ]:
# Flask App
from flask import Flask, render_template, request, redirect
from flask_ngrok import run_with_ngrok
from flask import current_app
import pytest

app = Flask(__name__)
run_with_ngrok(app)

@pytest.fixture
def client():
    with app.test_client() as client:
        with app.app_context():
            assert current_app.config["ENV"] == "production"
        yield client

def test_index_page(client):
   response = client.get('/')
   assert response.status_code == 200
   assert b'Welcome!' in response.data

# 1- Rendering Routes --------------------------------------------------------------------------------------------------------------------------------------------------------------------
@app.route("/")
@app.route("/page_principale")
def page():
   return render_template("page_principale.html")


@app.route("/ts")
@app.route("/index6.html")
def test():
  global data, list_events_test, custom_nn_model
  display_first = True
  custom_nn_model = False
  gc.collect()
  #Show test querries, first panel and list enventes
  return render_template("index6.html", test_en = data.test_en, test_ar = data.test_ar, display_first = display_first, custom_nn_model=custom_nn_model, list_events_test=list_events_test)



@app.route("/tr")
@app.route("/index5.html")
def train():
    return render_template("index5.html")


# 2- Data-submitted routes ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
@app.route("/submit_test", methods=['POST','GET'])
def submit_test():
  global var_radio_dl_model, var_radio_dl_model_old, data, custom_nn_model, user_dataset, list_events_test, custom_modeltype
  display_first = False # Display result similarity
  gc.collect()
  consolenext()

  #- Parse form inputs
  if request.form.get('ch_lang')=='on':
      langue='en'
      phrase_choisi=request.form.get('test_choix_en')
  else:
      langue='ar' 
      phrase_choisi=request.form.get('test_choix_ar')
  nbr_top_phrase = int(request.form.get('top_n_result'))
  var_radio_dl_model=request.form.get('radgroup_pre_model')

  # Load concerned dataset (can be user defined or our own)
  if custom_nn_model:
    data = user_dataset
  else:
    data = DatasetLoader()
    data.load(80/100, False)

  # Load selected Deep learning model (LSTM, GRU, BERT)
  if not custom_nn_model:
    load_deep_learning_model_pre(str(var_radio_dl_model))
  
  if var_radio_dl_model.startswith('BERT') or custom_modeltype=='bert':
     #Evaluation (BERT case)
     eval_result = evaluate_model_bert(phrase_choisi, nbr_top_phrase, custom_nn_model)
  else:
    #Evaluation (LSTM/GRU case)
    eval_result = evaluate_model_keras(phrase_choisi, nbr_top_phrase, langue, custom_nn_model)

  # Print test querries, evalResults(Similarities) and user selected phrase
  return render_template("index6.html",res=eval_result , test_en=data.test_en, test_ar=data.test_ar,phrase_choisi=phrase_choisi, display_first=display_first, custom_nn_model=custom_nn_model, list_events_test=list_events_test)


@app.route('/createNN', methods=['GET', 'POST'])
def createNN():
  global user_dataset, custom_modeltype,custom_nn_bert
  #-ALL: Get User's inputs
  nn_model_type, w2v_model_type , nbr_layers, nbr_in_layer1,\
  nbr_in_layer2, nbr_in_layer3, nbr_in_layer4, nbr_in_layer5,\
  dense, weights, dropout, activ_fun, learn_rate, momentum, decay_rate,\
  loss_fun, epochs, batchsize, optim, split_ratio, embeddings_length = parse_request(request)
  
  #-ALL: Load new instance of dataset STS for user with randomizing
  consolesep()
  consolesep()
  consoleln('Begin training user\'s custom model', 'lime')
  consoleln('Loading a new instance of STS dataset for User ...')
  user_dataset = DatasetLoader()
  user_dataset.load(float(split_ratio)/100, random = True)
  consoleln('STS dataset loaded!')
  
  console('There are')
  console(str(user_dataset.size) +'instances', 'pink')
  console('The split data goes:')
  console(str(user_dataset.train_size)+' for training', 'pink')
  console('and')
  consoleln(str(user_dataset.test_size)+'for testing', 'pink')

  #-LSTM|GRU: shares the sames structure (from KERAS), BERT is a transformer model
  if nn_model_type == 'LSTM' or nn_model_type == 'GRU':

    #-LSTM|GRU: Load User's W2v choice
    consoleln('Loading user\'s Word Embedings ...', 'yellow')
    load_w2v_model(w2v_model_type)

    # -LSTM|GRU:Preprocess dataset to create embeddings
    consoleln('Trying to embed trainset...', 'yellow')
    user_train_embd_en, user_train_embd_ar = preprocess(user_dataset.train_en, user_dataset.train_ar)
    consoleln('done!', 'green')
    consoleln('Trying to embed testset...', 'yellow')
    user_test_embd_en, user_test_embd_ar = preprocess(user_dataset.test_en, user_dataset.test_ar)
    consoleln('done!', 'green')

    #-LSTM|GRU: Building a custom neural network model
    consoleln('assembling User\'s custon neural network into shape...', 'yellow')

    #-LSTM|GRU: 1- init Loss function
    lstm_gru_loss_fun = get_keras_loss_fun(loss_fun)
        
    #-LSTM|GRU: 2- init Learning rate + it's decay rate
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=float(learn_rate),
        decay_steps=100000,
        decay_rate=float(decay_rate))

    #-LSTM|GRU: 3- init optimizer, note that only SGD and RMSPROP use momentum
    lstm_gru_optim = get_keras_optimizer(optim, lr_schedule, momentum)
    
    #-LSTM|GRU: 4- init weights
    weights_init = tf.keras.initializers.RandomUniform(minval=-float(weights), maxval=float(weights), seed=None)

    #-LSTM|GRU: Deleting models (if they exists)
    try: 
      del custom_nn
    except NameError:
      pass
    try: 
      del custom_nn_reverse
    except NameError:
      pass

    #-LSTM|GRU: Adding layers to the custom model
    custom_nn = Sequential()
    if nn_model_type == 'LSTM':
      custom_nn.add(LSTM(int(nbr_in_layer1), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
      custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>1:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(LSTM(int(nbr_in_layer2), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>2:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(LSTM(int(nbr_in_layer3), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>3:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(LSTM(int(nbr_in_layer4), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>4:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(LSTM(int(nbr_in_layer5), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      custom_nn.add(Dense(300, activation = activ_fun)) #Final dense layer
    elif nn_model_type == 'GRU':
      custom_nn.add(GRU(int(nbr_in_layer1), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
      custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>1:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(GRU(int(nbr_in_layer2), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>2:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(GRU(int(nbr_in_layer3), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>3:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(GRU(int(nbr_in_layer4), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      if int(nbr_layers)>4:
        custom_nn.add(Dense(int(dense)))
        custom_nn.add(GRU(int(nbr_in_layer5), input_shape=(maxlength,300), return_sequences=True, activation = activ_fun, kernel_initializer = weights_init))
        custom_nn.add(Dropout(float(dropout)))
      custom_nn.add(Dense(300, activation = activ_fun)) #Same final dense layer

    #-LSTM|GRU: Start training the custom neural network model
    custom_nn_reverse = tf.keras.models.clone_model(custom_nn)

    consoleln('Training the first'+nn_model_type+'model to learn and predict en- > ar', 'magenta')
    custom_nn.compile(optimizer=lstm_gru_optim, loss=lstm_gru_loss_fun ,metrics=['accuracy'])
    custom_nn.summary()
    custom_nn_history=custom_nn.fit(user_train_embd_en, user_train_embd_ar, validation_data=(user_test_embd_en, user_test_embd_ar), epochs=int(epochs), batch_size=int(batchsize)*len(user_train_embd_ar))
    
    consoleln('Training the second'+nn_model_type+'model to learn and predict ar- > en', 'magenta')
    custom_nn_reverse.compile(optimizer=lstm_gru_optim, loss=lstm_gru_loss_fun ,metrics=['accuracy'])
    custom_nn_reverse.summary()
    custom_nn_reverse_history=custom_nn.fit(user_train_embd_ar, user_train_embd_en, validation_data=(user_test_embd_ar, user_test_embd_en), epochs=int(epochs), batch_size=int(batchsize)*len(user_train_embd_ar))
    custom_modeltype='lstm_gru'
  else: # We creating a BERT Model
    consoleln('Downloading a new BERT model for user to train', 'yellow')
    from sentence_transformers import SentenceTransformer, models
    custom_nn_bert = SentenceTransformer('distiluse-base-multilingual-cased-v1')

    #-BERT: prepare training and test samples, structure is like this: 
    #[['arabic text1', 'corresponding english text1' 'score lbl'], [next], [ect], ...]
    train_samples, test_samples = [], []
      
    for i in range(len(data.train_en)):
      train_samples.append(InputExample(texts=[data.train_en[i], data.train_ar[i]], label = data.train_score[i]/5)) # normalise [0..1]
    train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=int(30)*data.train_size//100)

    for i in range(len(data.test_en)):
      test_samples.append(InputExample(texts=[data.test_en[i], data.test_ar[i]], label = data.test_score[i]/5))
    #dev_evaluator_sts = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

    #-BERT: Redy loss function
    bert_loss_function = get_bert_loss_fun(custom_nn_bert, 'CosineSimilarityLoss') 
    #-BERT: init activation function
    bert_activ_fun = get_bert_active_fun(activ_fun)
    #-BERT: Redy loss function
    bert_loss_function = get_bert_loss_fun(custom_nn_bert, loss_fun)
    
    #-BERT: Fine tuning the model
    custom_nn_bert.fit(
        train_objectives=[(train_dataloader, bert_loss_function)], 
        epochs=int(epochs), 
        show_progress_bar =True, 
        warmup_steps=1000,
        evaluation_steps=1000
    )

    #-BERT: Freeing GPU memory
    cuda.empty_cache()
    custom_modeltype = 'bert'

  display_first= True
  global custom_nn_model
  custom_nn_model = True
  return  render_template('/index6.html', test_ar=user_dataset.test_ar, test_en = user_dataset.test_en,  display_first = display_first, custom_nn_model=custom_nn_model, list_events_test=list_events_test)
  
# use this route to shutdown the server from ngrok side, without leaving open endpoints
@app.route('/sh')
def sh():
    shutdown_server = request.environ.get('werkzeug.server.shutdown')
    if shutdown_server is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    shutdown_server()
    return 'Server shutting down...'


if __name__ == '__main__':
  os.chdir('/content')
  allow_colab_print = True
  if first_run:
    first_run = False
  else:
    list_events_test=list()
    gc.collect()
  app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://176d-34-74-209-106.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/style5.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/page_principale.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/jquery-3.3.1.slim.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/solid.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/fontawesome.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/popper.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:04] "GET /static/bootstrap.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:05] "GET /static/img1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:05:05] "GET /static/img2.jpg HTTP

------------------------------------------------------- New Execution -----------------------------------------------------
Loading dataset without shuffling...
chargement d'un nouveau modèle de deep learning pré-entrainé par nous...
le nom code du modèle de deep learning est: LSTM_1
Le modèle deep learning n'a pas été changé, en garde le modèle ancien...
Basculer vers le model qui accespt la langue Anglais comme entrée
Pour fair l'évaluation la données d'entré d'utilisateur va être d'abord codifié en word embeddings...
comparaison de nombre des mots de la plus grande phrase dans la liste des phrase d'anglais (15) avec celui de la liste  d'arabe (0)  Et avec le Mexlength minimum défini par  nous (20) cette comparaison va nous données une idéé sur la taille de paddings qu'on va l'utilisé après Le (MaxLength) a été choisi (20) , Les phrases seront étendues avec des paddings jusqu'ils sont tousde même taille.
--------------------------------------------------------------------------------

...

INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:18:33] "POST /submit_test HTTP/1.1" 200 -


Codification du l'entré de l'utilisateur en embeddings
Calcule de la similarité entre les embeddings de dataset avec celui de l'utilisateur
Normalisation les valeurs de similartés entre [0,1] et les faire le trié selon l'odre decroissant


INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:19:37] "POST /submit_test HTTP/1.1" 200 -


------------------------------------------------------- New Execution -----------------------------------------------------
Loading dataset without shuffling...
chargement d'un nouveau modèle de deep learning pré-entrainé par nous...
le nom code du modèle de deep learning est: BERT_A
Le modèle deep learning n'a pas été changé, en garde le modèle ancien...
Les embeddings du dataset pour le modèle BERT existent deja, pas la peine de les créer à nouveau
Codification du l'entré de l'utilisateur en embeddings
Calcule de la similarité entre les embeddings de dataset avec celui de l'utilisateur
Normalisation les valeurs de similartés entre [0,1] et les faire le trié selon l'odre decroissant


INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:19:45] "GET /index5.html HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Sep/2022 07:20:09] "GET /index6.html HTTP/1.1" 200 -


#Failsafe (Do not use)

In [ ]:
%reset # del all vars

In [ ]:
# In case you see a log like this, run this celle to reset the 'request' variable and tp delete the current app.
"""
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/local.py", line 347, in __getattr__
    return getattr(self._get_current_object(), name)
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/local.py", line 306, in _get_current_object
    return self.__local()
  File "/usr/local/lib/python3.7/dist-packages/flask/globals.py", line 38, in _lookup_req_object
    raise RuntimeError(_request_ctx_err_msg)
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request.  Consult the documentation on testing for
information about how to avoid this problem.
"""
try:
  del request
except NameError:
  pass
try:
  del app
except NameError:
  pass

In [ ]:
# Release GPU Memory
import gc
gc.collect()
cuda.empty_cache()

# BERT, going anew

In [ ]:
"""
#-BERT: Download bert's embeddings
bert_word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=int(embeddings_length))

#-BERT: init internal module for bert model
pooling_model = models.Pooling(bert_word_embedding_model.get_word_embedding_dimension())

#-BERT: init activation function
bert_activ_fun = get_bert_active_fun(activ_fun)

#-BERT: link everything into dense
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=int(embeddings_length), 
    activation_function=bert_activ_fun
    )

#-BERT: Create a new BERT model containing all modules previously created
bert_model = SentenceTransformer(
    modules=[bert_word_embedding_model, pooling_model, dense_model], 
    device='cuda:0'
    )
"""

from sentence_transformers import SentenceTransformer, models
modelB = SentenceTransformer('distiluse-base-multilingual-cased-v1')


#-BERT: prepare training and test samples, structure is like this [['arabic text1', 'corresponding english text1' 'similarity'], [next], [ect], ...]
train_samples, test_samples = [], []
for i in range(len(data.train_en)):
  train_samples.append(InputExample(texts=[data.train_en[i], data.train_ar[i]], label = data.train_score[i]/5)) # normalise [0..1]
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=int(30)*data.train_size//100)

for i in range(len(data.test_en)):
  test_samples.append(InputExample(texts=[data.test_en[i], data.test_ar[i]], label = data.test_score[i]/5))
#dev_evaluator_sts = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

#-BERT: Redy loss function
bert_loss_function = get_bert_loss_fun(modelB, 'CosineSimilarityLoss') 

#-BERT: Fine tuning the model
modelB.fit(
    train_objectives=[(train_dataloader, bert_loss_function)], 
    epochs=int(100), 
    show_progress_bar =True, 
    warmup_steps=1000,
    evaluation_steps=1000
)

#-BERT: Freeing GPU memory
cuda.empty_cache()

##save model bert


In [ ]:
os.chdir('/content/drive/MyDrive/Colab Notebooks/Web/pre_entrainer')
modelB.save("modelbert_100ep_batch30.h5")

In [ ]:
modelB = SentenceTransformer('/content/drive/MyDrive/Colab Notebooks/Web/pre_entrainer/modelbert_100ep_batch30.h5/')

## Calculate similarity

In [ ]:
from sentence_transformers import SentenceTransformer, util

# Two lists of sentences
sentences1 = ['two men are joying']

sentences2 = ['رجلان يضحكان']

#Compute embedding for both lists
embeddings1 = custom_nn_bert.encode(data.lines_ar, convert_to_tensor=True)
embeddings2 = custom_nn_bert.encode(data.lines_en, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(data.lines_ar)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(data.lines_ar[i], data.lines_en[i], cosine_scores[i][i]))

AttributeError: ignored

In [ ]:
from sentence_transformers import SentenceTransformer, util

use_input = ['رجلان يضحكان ']

#Compute embedding for both lists
bert_embeddings_ar = modelB.encode(data.lines_ar, convert_to_tensor=True)
bert_embeddings_en = modelB.encode(data.lines_en, convert_to_tensor=True)
bert_user_embeddings = modelB.encode(use_input, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores_ar = util.cos_sim(bert_embeddings_ar, bert_user_embeddings)
cosine_scores_en = util.cos_sim(bert_embeddings_en, bert_user_embeddings)
cosine_scores = torch.cat((cosine_scores_ar, cosine_scores_en))
cosine_scores, _ = torch.sort(cosine_scores)
#T = torch.cat((T1,T2,T3))
#v = torch.sort(T)

printdata = data.lines_ar+data.lines_en
print( len(cosine_scores))
#Output the pairs with their score
for i in range(0, len(cosine_scores), 2):
    print("{} \t\t {} \t\t Score: {:.4f}".format(printdata[i], use_input[0], cosine_scores[i][0]))

"""
#Output the pairs with their score
for i in range(len(data.lines_en)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(data.lines_en[i], use_input[0], cosine_scores_en[i][0]))"""

In [ ]:

cosine_scores_ar = util.cos_sim(bert_embeddings_ar, bert_user_embeddings)
cosine_scores_en = util.cos_sim(bert_embeddings_en, bert_user_embeddings)

#Concat two tensors results ar+en
concated = torch.cat((cosine_scores_ar, cosine_scores_en))

#Sort while keeping in mind swaping the order of dataset lines too
cosine_scores = sorted(zip(concated, data.lines_ar+data.lines_en), reverse = True)

# normalizing similarity into [0, 1] instead of [-1, 1] & rewriting evrything into a good looking list
cosine_scores = [(y, float("{:.4f}".format((x[0].item()+1)/2))) for x,y in cosine_scores]

#Assertion check
for x in cosine_scores:
  assert float(x[1])>=0 and float(x[1]) <=1




## Evaluator

In [ ]:
evaluators = []

"""
test_evaluator = evaluation.EmbeddingSimilarityEvaluator(bert_embeddings_en, bert_embeddings_ar, data.lines_score, batch_size = 20, name='AR-EN', show_progress_bar=True)
evaluators.append(test_evaluator)
"""
test_evaluator = evaluation.EmbeddingSimilarityEvaluator(data.lines_ar, data.lines_en, data.lines_score, batch_size = 30, name='resultsss', show_progress_bar=False, write_csv = True)


modelB.fit(
    evaluator=test_evaluator,
    train_objectives=[(train_dataloader, bert_loss_function)], 
    epochs=int(100), 
    show_progress_bar =True, 
    warmup_steps=1000,
    evaluation_steps=1000
)



In [ ]:

os.chdir("/content")
test_evaluator = evaluation.EmbeddingSimilarityEvaluator(data.lines_ar, data.lines_en, data.lines_score, batch_size = 30, name='AR-EN', show_progress_bar=True, write_csv = True)
#def __call__(self, model, output_path: str = None, epoch: int = -1, steps: int = -1) 
for i in range(100):
  test_evaluator(modelB,'',i, 100 )
